In [1]:
import pycbc.noise
import pycbc.psd
import pycbc.filter
from pycbc.types import timeseries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from scipy import signal
import matplotlib.pyplot as plt
from numpy.random import uniform, randint
import pylab
from pycbc.filter import sigma
from pycbc.waveform import get_td_waveform
from pycbc.psd import interpolate
from pycbc.types.timeseries import load_timeseries

#import false signal functions
from ipynb.fs.full.false_signals import random_false_sig, flip_gw

PyCBC.libutils: pkg-config call failed, setting NO_PKGCONFIG=1


# functions

In [2]:
# Generate 1 seconds of noise at 4096 Hz, randomized, unscaled

def noise_d():
    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    return ts

In [3]:
# Generate gravitational waveform with specified mass parameters.
# Other arguments fixed
# return the plus polarization

# mass range does not need to be restricted any more.
# for signal length < 1, it is shifted. 
# for signal length > 1, the waveform is cropped on the left to keep the right-hand side with the merger moement.
# For the ease of merging, the start_time is reset to 0 and total length is 1

def gw_d(m1,m2,d):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          distance=d,
                          delta_t=1.0/4096,
                          f_lower=20)
    
    # randomly shift
    # To fit the whole waveform in the 1 second window, need to specify a cyclic shift range. 
    # 0.08 act as buffer since signal still oscillates a little after merging.
    sigtime = abs(hp.start_time) + 0.08
    
    if sigtime < 1:
        # resize to window of 1 second
        hp.resize(4096)
        shift_range = 1 - sigtime
        shifted = hp.cyclic_time_shift(uniform(0.0,shift_range))
    else:
        totaltime = hp.duration
        hp.start_time = 0
        hp = hp.crop(0, totaltime - sigtime)
        shifted = hp.crop(hp.duration-1,0)
    
    # reset start time
    shifted.start_time = 0
    
    return shifted

In [4]:
def burried_gw_d(m1,m2,d):
    sig = gw_d(m1,m2,d)
    noi = noise_d()
    
    return noi + sig

In [5]:
def burried_false_d(mag):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise_d()
    
    return noi + fal

In [6]:
def burried_flip_d(m1,m2,d):
    hp = gw_d(m1,m2,d)
    flipped = flip_gw(hp)
    noi = noise_d()
    
    return noi + flipped

In [7]:
# return partial_burried_false_signals 
# magnitude and snr same as burried_false
# percentage: keep 0 to 1 of the waveform, randomly chosen from anypart of the original waveform

def partial_burried_false_d(mag,percentage):
    fal = random_false_sig(mag)
    fal.resize(4096)
    noi = noise_d()
    
    # left + right = 1 - percentage
    left = uniform(0,1-percentage)
    right = (1-percentage)-left
    fal = fal.crop(left, right)
    
    # complete to 4096 data
    zeros = 4096 - len(fal)
    left_zeros = randint(zeros+1)
    right_zeros = zeros - left_zeros 
    fal.append_zeros(right_zeros)
    fal.prepend_zeros(left_zeros)
    
    fal.start_time = 0
    
    return fal+noi

In [8]:
# return partial_burried_signals 
# The partial signal generated by cropping off left side and append zeros on right
# In this way, the merger moement is included. 
# The purpose is that the net can detect the gw as long as merger moment included and a signal of 50%-100% remains.
# percentage means the part kept

def partial_burried_gw_d(m1,m2,d,percentage):
    hp,hc=get_td_waveform(approximant="SEOBNRv4_opt",
                          mass1=m1,    
                          mass2=m2,
                          distance = d,
                          delta_t=1.0/4096,
                          f_lower=20)

    # crop to only keep signal
    sigtime = abs(hp.start_time) + 0.08
    totaltime = hp.duration
    hp.start_time = 0
    hp = hp.crop(0, totaltime - sigtime)
    
    # if really long only keep 1 sec
    if sigtime > 1:
        hp = hp.crop(hp.duration-1,0)
        left = 1-percentage
    else:
        left = (1-percentage) * sigtime
    # now keep only certain percentage from right end side
    hp.start_time = 0
    
    hp = hp.crop(left, 0)
    
    # complete to 4096 data
    zeros = 4096 - len(hp)
    hp.append_zeros(zeros)
    
    # burry in noise
    noi = noise_d()    
    hp.start_time = 0
        
    return noi + hp

# check SNRs

In [39]:
def output_snr(m1,m2,d):

    S = gw_d(m1,m2,d)

    # psd
    flow = 20.0
    delta_f = 1.0 / 16
    flen = int(2048 / delta_f) + 1
    psd = pycbc.psd.aLIGOZeroDetHighPower(flen, delta_f, flow)
    
    # un-scaled noise
    delta_t = 1.0 / 4096
    tsamples = int(1.0 / delta_t)
    ts = pycbc.noise.noise_from_psd(tsamples, delta_t, psd)
    
    new_psd = pycbc.psd.estimate.interpolate(psd, S.delta_f)  # First interpolate psd to desire delta_f
    return sigma(S, psd=new_psd, low_frequency_cutoff=20.0)


In [49]:
# lowest and highest SNRs possible considering mass range 10-100 and distance range 100-2000

print("lowest snr with m1,m2 10 and distance 2000:", output_snr(10,10,2000))
print("highest snr with m1,m2 100 and distance 100:", output_snr(100,100,100))

lowest snr with m1,m2 10 and distance 2000: 7.166659635243008
highest snr with m1,m2 100 and distance 100: 953.5509668867712
